In [2]:
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader
loader = LLMSherpaFileLoader(
    file_path="/home/ubuntu/T_RAG/testing/2407.13734v1.pdf",
    new_indent_parser=True,
    apply_ocr=True,
    strategy="html",
    llmsherpa_api_url="http://localhost:5010/api/parseDocument?renderFormat=all",
)
docs = loader.load()

In [3]:
docs

[Document(metadata={'source': '/home/ubuntu/T_RAG/testing/2407.13734v1.pdf'}, page_content='<html><h1>Understanding      Reinforcement      Learning-Based</h1><h1>Fine-Tuning      of      Diffusion      Models:      A      Tutorial      and      Review</h1><p>Masatoshi      Uehara*!,      Yulai      Zhao\\\\\\\'*,      Tommaso      Biancalani!,      and      Sergey      Levine?</p><p>‘Genentech</p><li>*Princeton      University</li><p>>University      of      California,      Berkeley</p><h2>July      19,      2024</h2><h2>Abstract</h2><p> \n \n  This      tutorial      provides   \n \na      comprehensive      survey      of      methods      for      fine-tuning      diffusion      models  \n \n  to      optimize      downstream      reward      functions.\nWhile      diffusion      models      are      widely      known      to      provide  \n \n  excellent      generative      modeling      capability,      practical      applications      in      domains      such      as      bi

In [8]:
docs

[Document(metadata={'source': '/home/ubuntu/T_RAG/testing/2407.13734v1.pdf'}, page_content='<html><h1>Understanding      Reinforcement      Learning-Based</h1><h1>Fine-Tuning      of      Diffusion      Models:      A      Tutorial      and      Review</h1><p>Masatoshi      Uehara*!,      Yulai      Zhao\\\\\\\'*,      Tommaso      Biancalani!,      and      Sergey      Levine?</p><p>‘Genentech</p><li>*Princeton      University</li><p>>University      of      California,      Berkeley</p><h2>July      19,      2024</h2><h2>Abstract</h2><p> \n \n  This      tutorial      provides   \n \na      comprehensive      survey      of      methods      for      fine-tuning      diffusion      models  \n \n  to      optimize      downstream      reward      functions.\nWhile      diffusion      models      are      widely      known      to      provide  \n \n  excellent      generative      modeling      capability,      practical      applications      in      domains      such      as      bi

In [10]:
docs[0]

TypeError: 'Document' object is not subscriptable